Importando as bibliotecas necessárias

In [ ]:
import moviepy.editor as mpe
import numpy as np

Importando o vídeo bruto do projeto

In [2]:
video = mpe.VideoFileClip("video_projeto.mp4")

Pegando os valores de altura e largura do vídeo para utilizar no parâmetro "shape"

In [4]:
print('Tamanho:', video.size) ## ou video.h e video.w

Tamanho: [1920, 1080]


Função que inverte o vídeo em X. Feito assim para economizar tempo na renderização

In [5]:
def inverte_frame(frame):
    new_frame = np.zeros(shape=(1080, 1920, 3), dtype=np.uint32)
    new_frame[::1, ::-1] = frame
    return new_frame

Este trecho inverte efetivamente o vídeo a cada 20 segundos. Todos os trechos são armazenados em uma lista. A cada iteração, caso o valor de inicio seja um múltiplo de 20, inverte os frames. Após isso, todos os frames na lista são concatenados

In [6]:
duracao = video.duration
partes = []
for inicio in range(0, int(duracao), 20):
    fim = min(inicio + 20, duracao)
    parte = video.subclip(inicio, fim)
    
    if (inicio // 20) % 2 == 0:
        parte = parte.fl_image(inverte_frame)
        
    partes.append(parte)

video_invertido = mpe.concatenate_videoclips(partes)
video_invertido.write_videofile("video_invertido.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_invertido.mp4.
MoviePy - Writing audio in video_invertidoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_invertido.mp4



Moviepy - Done !
Moviepy - video ready video_invertido.mp4


Funções que alteram o áudio. Uma máscara é usada para saber o intervalo onde se deve alterar gradativamente o volume e o fator de multiplicação é calculado. O fator de multiplicação mostra a porcentagem a que o áudio será reduzido. Quando o vídeo chegar em 10 segundos, o áudio será zerado

In [ ]:
def volume(t, audio_shape):
    #a máscara que será usada para modificar o volume
    mask = t >= duracao_total - 10

    #intervalo de 30 segundos entre uma alteração de volume e outra
    intervals = t // 30
    
    #o fator de multiplicação
    factors = 1 - 0.1 * intervals
    
    #caso chegue nos 10 minutos finais
    factors[mask] = 0.0
    
    #retorna o fator de multiplicação para ser usado no volume do audio
    return np.tile(np.maximum(factors, 0.0), (audio_shape[1], 1)).T

def aplicar_volume(get_audio_frame, t):
    #recebe o frame de audio e o fato de multiplicação
    audio_frame = get_audio_frame(t)
    fator = volume(t, audio_frame.shape)
    
    # Verificando se os tipos de dados são compatíveis
    if audio_frame.dtype != fator.dtype:
        fator = fator.astype(audio_frame.dtype)
    
    #retorna o novo valor de audio
    return audio_frame * fator


Aplicando alterações no audio

In [ ]:
duracao_total = video_invertido.duration
audio = video_invertido.audio.fl(aplicar_volume)
video_invertido_com_audio = video_invertido.set_audio(audio)

In [21]:
video_invertido_com_audio.write_videofile("video_invertido_com_audio.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_invertido_com_audio.mp4.
MoviePy - Writing audio in video_invertido_com_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_invertido_com_audio.mp4



Moviepy - Done !
Moviepy - video ready video_invertido_com_audio.mp4


Neste trecho, o vídeo é cortado. Reproduzido normalmente até os 60 segundos, depois avaça para os 80 segundos, depois volta para os 20 segundos que foram cortados

In [16]:
segmento_cortado = video_invertido_com_audio.subclip(60, 80)
video_final = mpe.concatenate_videoclips([
    video_invertido_com_audio.subclip(0, 60),
    video_invertido_com_audio.subclip(80, video_invertido_com_audio.duration),
    segmento_cortado
])

In [29]:
video_final.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
